### Setup

Enable parallel computing.

In [ ]:
using Distributed
addprocs(12);

In [ ]:
@everywhere using Distributions, StatsBase, Distances, LinearAlgebra, SharedArrays, JuliennedArrays, Random, DelimitedFiles

@everywhere begin
    const numb_hyp = 11
    const numb_agent = 50
    const numb_toss = 500
    const likelihood_heads = range(0, stop=1, length=numb_hyp)
    const likelihood_tails = range(1, stop=0, length=numb_hyp)
    const numb_sim = 50
end

### Data generation function

The following function generates sequences of coin tosses, one per agent, where the bias of the coin is given by `bias` and the length of the sequences by `numb_toss`.

In [ ]:
@everywhere function generate_data(bias::Float64)
    sim_dat = Array{Int64,2}(undef, numb_agent, numb_toss)
    for i in 1:numb_agent
        sim_dat[i, :] = rand(Bernoulli(bias), numb_toss)
    end
    return sim_dat
end

### Update rules

These are the update rules to be compared. The first is Bayes' rule, which has `bonus` as a kind of dummy parameter, to make sure it has to right form for the evolutionary computing to be carried out later. The other rules can in fact also do duty as Bayes' rule (which is the limiting case of each of them, obtained by setting `bonus = 0`).

In [ ]:
# Bayes' rule
@everywhere function b_upd(probs::Array{Float64,1}, dat::Array{Int64,1}, toss_num::Int64, bonus::Float64)
    if dat[toss_num] == 1
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end
end

# EXPL
@everywhere function expl_upd(probs::Array{Float64,1}, dat::Array{Int64,1}, toss_num::Int64, bonus::Float64)
    val::Float64 = mean(dat[1:toss_num]) * (numb_hyp - 1.0) + 1.0
    vec::Array{Float64,1} = if dat[toss_num] == 1
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end
    if val % 1 == .5
        vec[floor(Int, val)] += bonus*0.5
        vec[ceil(Int, val)] += bonus*0.5
    else
        vec[round(Int, val)] += bonus
    end
    return vec / (1.0 + bonus)
end

#Good's rule; with λ=2 (default value), we obtain the rule L2 from Douven & Schupbach, 2015 (Frontiers paper)
@everywhere function good_bonus(probs::Array{Float64,1}, res::Int64, λ=2.0)
    pE::Float64 = res == 1 ? dot(probs, likelihood_heads) : dot(probs, likelihood_tails)
    gb::Array{Float64,1} = res == 1 ? log.(likelihood_heads ./ pE) : log.(likelihood_tails ./ pE)
    function rsc(i)
        if i >= 0
            1 - exp(2λ^2 * -i^2)
        else
            -1 + exp(2λ^2 * -i^2)
        end
    end
    return map(rsc, gb)
end

# γ is the proportion of the probability that gets added as a bonus, so necessarily γ ⩽ 1
@everywhere function good_upd(probs::Array{Float64, 1}, dat::Array{Int64,1}, toss_num::Int64, γ::Float64)
    res::Int64 = dat[toss_num]
    probvec::Array{Float64,1} = if res == 1
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end
    goodvec::Array{Float64,1} = probvec + γ .* (probvec .* good_bonus(probs, res))
    return goodvec / sum(goodvec)
end


# Popper's rule; in pop_upd, γ is the proportion of the probability that gets added as a bonus, so necessarily γ ⩽ 1
@everywhere function pop_bonus(probs::Array{Float64,1}, res::Int64)
    pE::Float64 = res == 1 ? dot(probs, likelihood_heads) : dot(probs, likelihood_tails)
    pb::Array{Float64,1} = res == 1 ? (likelihood_heads .- pE) ./ (likelihood_heads .+ pE) : (likelihood_tails .- pE) ./ (likelihood_tails .+ pE)
end

@everywhere function pop_upd(probs::Array{Float64,1}, dat::Array{Int64,1}, toss_num::Int, γ::Float64)
    res::Int64 = dat[toss_num]
    probvec::Array{Float64,1} = if res == 1
        @. (probs * likelihood_heads) / $dot(probs, likelihood_heads)
    else
        @. (probs * likelihood_tails) / $dot(probs, likelihood_tails)
    end
    popvec::Array{Float64,1} = probvec + γ .* (probvec .* pop_bonus(probs, res))
    return popvec / sum(popvec)
end

The following function is meant to generalize the simulations reported in Douven & Wenmackers, 2017. Now we can compare more rules than just Bayes' rule and EXPL, as well as other ways of measuring distances among probability functions and other ways of averaging probabilities.

In [ ]:
@everywhere function social_learning(rule::Function, dist::DataType, averaging, bonus::Float64, ϵ::Float64, sdat::Array{Int64,2})
    UPD = Array{Float64,3}(undef, numb_hyp, numb_agent, numb_toss + 1)
    UPD[:, :, 1] = repeat(fill(1/numb_hyp, numb_hyp), 1, numb_agent)
    PROB = Array{Float64,2}(undef, numb_hyp, numb_agent)
    f(x::Float64) = (x<=ϵ)::Bool
    @inbounds for t in 1:numb_toss
        for i in 1:numb_agent
            PROB[:, i] = rule(UPD[:, i, t], sdat[i, :], t, bonus)
        end
        prob_dist::Array{Float64,2} = pairwise(dist(), PROB, dims=2)
        peers::Array{Bool,2} = map(f, prob_dist)
        if averaging == mean
            @inbounds for i::Int in 1:numb_agent
               v = @view PROB[:, peers[i, :]]
               UPD[:, i, t + 1] = mean(v', dims=1)
            end
        else
            @inbounds for i::Int in 1:numb_agent
               q = map(averaging, Slices(PROB[:, peers[i, :]], False(), True()))
               UPD[:, i, t + 1] = q / sum(q)
            end
        end
    end
    return UPD
end

### Accuracy and speed

We will be looking at trade-offs between accuracy and speed of convergence. A standard way of measuring the former involves the so-called Brier score.

In [ ]:
@everywhere function brierScore(mat::Array{Float64}, agent::Int, toss::Int, hyp::Int)
    (1 - mat[:, agent, toss][hyp])^2 + sum(deleteat!(mat[:, agent, toss], hyp).^2)
end

In [ ]:
@everywhere function totalBrier(mat::Array{Float64,3}, hyp::Int)
    tot = 0.0
    for i in 1:numb_agent, j in 1:numb_toss
        tot += brierScore(mat, i, j, hyp)
    end
    return tot
end

In [ ]:
@everywhere function first_above(mat::Array{Float64,3}, hyp::Int, θ=.9)
    g(toss) = sum(mat[hyp, : , toss] .> θ)
    ff = findfirst(x->x>div(numb_agent, 2), map(g, 1:numb_toss + 1))
    out = ff == nothing ? numb_toss : ff
end

### Evolutionary computing

The scoring function, which scores groups on the basis of averages taken over 25 rounds.

In [ ]:
@everywhere function avScore(rule::Function, dist::DataType, averaging, bonus::Float64, ϵ::Float64)
    br_sc = Array{Float64,1}(undef, 25)
    tr_sc = Array{Int64,1}(undef, 25)
    for i in 1:25
        rand_hyp::Int64 = rand(1:numb_hyp)
        sdat::Array{Int64,2} = generate_data((rand_hyp - 1)/(numb_hyp - 1))
        simres::Array{Float64,3} = social_learning(rule, dist, averaging, bonus, ϵ, sdat)
        br_sc[i] = totalBrier(simres, rand_hyp)
        tr_sc[i] = first_above(simres, rand_hyp)
    end
    return mean(br_sc), mean(tr_sc)
end

Function for generating offspring.

In [ ]:
@everywhere function offspring(a::Int64, b::Int64, mat::Array{Any,2})
    r::Int64 = rand(1:3)
    s::Vector{Int64} = sample(1:3, r, replace=false, ordered=true)
    sd::Vector{Int64} = setdiff(1:3, s)
    c = Vector{Any}(undef, 5)
    c[s] = mat[a, :][s]
    c[sd] = mat[b, :][sd]
    c[4] = c[1] == b_upd ? 0.0 : rand(truncated(Normal(mean([mat[a, 4], mat[b, 4]]), std(mat[:, 4])), 0.0, 1.0))
    c[5] = rand(truncated(Normal(mean([mat[a, 5], mat[b, 5]]), std(mat[:, 5])), 0.0, 1.0))
    return c::Vector{Any}
end

Function for creating a new generation by selecting the 50 percent fittest and having them generate offspring.

In [ ]:
@everywhere function new_generation(mat::Array{Any,2}, w1::Int64, w2::Int64)
    tbs = Vector{Float64}(undef, 36) # total Brier score
    fat = Vector{Float64}(undef, 36) # first toss after which majority assigns probability > θ to truth
    @inbounds for i in 1:36
        tbs[i], fat[i] = avScore(mat[i,:]...)
    end
    ord_lst::Array{Bool,1} = ordinalrank(w1*ordinalrank(tbs) .+ w2*ordinalrank(fat)) .< 19
    cp_lst::Array{Any,2} = mat[ord_lst, :]
    cp_bonus_std::Float64 = std(cp_lst[:, 4])
    cp_eps_std::Float64 = std(cp_lst[:, 5])
    smp1::Vector{Int64} = shuffle(1:18)
    smp2::Vector{Int64} = shuffle(1:18)
    offspr = Array{Any,2}(undef, 18, 5)
    @inbounds for i in 1:18
        offspr[i, :] = offspring(smp1[i], smp2[i], cp_lst)
    end
    return vcat(cp_lst, offspr), mean(tbs), std(tbs), minimum(tbs), mean(fat), std(fat), minimum(fat)
end

The main function; results are written to text files in the same directory in which the notebook is located. (The function weighs speed and accuracy equally.)

In [ ]:
@everywhere function single_run(r)
    rule_id = vcat(repeat([b_upd], 9), repeat([expl_upd], 9), repeat([good_upd], 9), repeat([pop_upd], 9))
    c_vals = vcat(repeat([0.0], 9), rand(Uniform(), 27)) # bonus values are randomly chosen from [0, 1]
    eps_vals = rand(Uniform(), 36) # same for ϵ
    dist_id = repeat(vcat(repeat([Euclidean], 3), repeat([Cityblock], 3), repeat([KLDivergence], 3)), 4)
    av_id = repeat([mean, geomean, harmmean], 12)
    pop_start = hcat(rule_id, dist_id, av_id, c_vals, eps_vals)

    run(`mkdir gen_data$r`)
    open("gen_data$r/gen1.txt", "w") do io
        writedlm(io, pop_start)
    end

    brsc = Array{Float64,1}(undef, 49)
    fab = Array{Float64,1}(undef, 49)
    brsc_std = Array{Float64,1}(undef, 49)
    fab_std = Array{Float64,1}(undef, 49)
    brsc_min = Array{Float64,1}(undef, 49)
    fab_min = Array{Float64,1}(undef, 49)
    old = pop_start
    for i in 2:50
        ng = new_generation(old, 1, 1)
        new = ng[1]
        brsc[i - 1] = ng[2]
        brsc_std[i - 1] = ng[3]
        brsc_min[i - 1] = ng[4]
        fab[i - 1] = ng[5]
        fab_std[i - 1] = ng[6]
        fab_min[i - 1] = ng[7]
        open("gen_data$r/gen$i.txt", "w") do io
            writedlm(io, new)
        end
        old .= new
        new = nothing
        GC.gc()
    end
    open("gen_data$r/BR.txt", "w") do io
        writedlm(io, brsc)
    end
    open("gen_data$r/BRstd.txt", "w") do io
        writedlm(io, brsc_std)
    end
    open("gen_data$r/BRmin.txt", "w") do io
        writedlm(io, brsc_min)
    end
    open("gen_data$r/FA.txt", "w") do io
        writedlm(io, fab)
    end
    open("gen_data$r/FAstd.txt", "w") do io
        writedlm(io, fab_std)
    end
    open("gen_data$r/FAmin.txt", "w") do io
        writedlm(io, fab_min)
    end
end

We run the function 15 times, in parallel.

In [ ]:
pmap(single_run, 1:15);

Load the data concerning the final generation in each run.

In [ ]:
final_gens = Array{Any,3}(undef, 36, 5, 15)

for i in 1:15
    final_gens[:, :, i] = readdlm("gen_data_new$i/gen50.txt")
end

Count of update rules represented in last generations.

In [ ]:
countmap(final_gens[:, 1, :])

Same for distance measures and, respectively, pooling methods.

In [ ]:
countmap(final_gens[:, 2, :])

In [ ]:
countmap(final_gens[:, 3, :])

Mean and standard deviation of explanation bonus of best groups in last generations. 

In [ ]:
mean(final_gens[1, 4, :][:]), std(final_gens[1, 4, :][:]) 

Same for $\epsilon$ values.

In [ ]:
mean(final_gens[1, 5, :][:]), std(final_gens[1, 5, :][:]) 

Counts of types of update rule, types of metric, and types of pooling method -- to create figures.

In [ ]:
all_gens = Array{Any,3}(undef, 36, 5, 50)

for i in 1:50
    all_gens[:, :, i] = readdlm("gen_data_new1/gen$i.txt")
end

In [ ]:
ks = [keys(countmap(all_gens[:, 1, i])) for i in 1:50]
vls = [values(countmap(all_gens[:, 1, i])) for i in 1:50]

bayes = Vector{Int64}(undef, 50)
good = Vector{Int64}(undef, 50)
expl = Vector{Int64}(undef, 50)
pop = Vector{Int64}(undef, 50)

for i in 1:50
    bayes[i] = Int(collect(vls[i])[findall(x->x=="b_upd", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="b_upd", collect(ks[i]))][1])
    good[i] = Int(collect(vls[i])[findall(x->x=="good_upd", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="good_upd", collect(ks[i]))][1])
    expl[i] = Int(collect(vls[i])[findall(x->x=="expl_upd", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="expl_upd", collect(ks[i]))][1])
    pop[i] = Int(collect(vls[i])[findall(x->x=="pop_upd", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="pop_upd", collect(ks[i]))][1])
end

In [ ]:
rule_types = hcat(bayes, expl, good, pop);

In [ ]:
open("rule_types1.txt", "w") do io
    writedlm(io, rule_types)
end

In [ ]:
ks = [keys(countmap(all_gens[:, 2, i])) for i in 1:50]
vls = [values(countmap(all_gens[:, 2, i])) for i in 1:50]

eucl = Vector{Int64}(undef, 50)
city = Vector{Int64}(undef, 50)
kl = Vector{Int64}(undef, 50)

for i in 1:50
    eucl[i] = Int(collect(vls[i])[findall(x->x=="Euclidean", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="Euclidean", collect(ks[i]))][1])
    city[i] = Int(collect(vls[i])[findall(x->x=="Cityblock", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="Cityblock", collect(ks[i]))][1])
    kl[i] = Int(collect(vls[i])[findall(x->x=="KLDivergence", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="KLDivergence", collect(ks[i]))][1])
end

In [ ]:
dist_types = hcat(eucl, kl, city);

In [ ]:
open("dist_types1.txt", "w") do io
    writedlm(io, dist_types)
end

In [ ]:
ks = [keys(countmap(all_gens[:, 3, i])) for i in 1:50]
vls = [values(countmap(all_gens[:, 3, i])) for i in 1:50]

mn = Vector{Int64}(undef, 50)
gmn = Vector{Int64}(undef, 50)
hmn = Vector{Int64}(undef, 50)

for i in 1:50
    mn[i] = Int(collect(vls[i])[findall(x->x=="Statistics.mean", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="Statistics.mean", collect(ks[i]))][1])
    gmn[i] = Int(collect(vls[i])[findall(x->x=="StatsBase.geomean", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="StatsBase.geomean", collect(ks[i]))][1])
    hmn[i] = Int(collect(vls[i])[findall(x->x=="StatsBase.harmmean", collect(ks[i]))] != [] && collect(vls[i])[findall(x->x=="StatsBase.harmmean", collect(ks[i]))][1])
end

In [ ]:
av_types = hcat(mn, gmn, hmn);

In [ ]:
open("av_types1.txt", "w") do io
    writedlm(io, av_types)
end

Mean and standard deviation of bonus values.

In [ ]:
c_mn = Float64[]
c_std = Float64[]

for i in 1:50
    lst = all_gens[:, :, i]
    m, s = mean_and_std(convert(Array{Float64,1}, lst[:, 4]))
    push!(c_mn, m)
    push!(c_std, s)
end

In [ ]:
bonus_gen = hcat(round.(c_mn, digits=6), round.(c_std, digits=6));

In [ ]:
open("gen_bonus1.txt", "w") do io
    writedlm(io, bonus_gen)
end

Same for $\epsilon$ values.

In [ ]:
eps_mn = Float64[]
eps_std = Float64[]

for i in 1:50
    lst = all_gens[:, :, i]
    m, s = mean_and_std(convert(Array{Float64,1}, lst[:, 5]))
    push!(eps_mn, m)
    push!(eps_std, s)
end

In [ ]:
eps_gen = hcat(round.(eps_mn, digits=6), round.(eps_std, digits=6));

In [ ]:
open("eps1.txt", "w") do io
    writedlm(io, eps_gen)
end

In [ ]:
as = Array{Float64,2}(undef, 10, 2)
for i in 1:10
    as[i, :] = [avScore(b_upd, Cityblock, mean, 0.0, 0.0)...]
end

In [ ]:
vcat(mean(as, dims=1), std(as, dims=1))

In [ ]:
as1 = Array{Float64,2}(undef, 10, 2)
for i in 1:10
    as1[i, :] = [avScore(expl_upd, Euclidean, harmmean, 0.95, 0.89)...]
end

In [ ]:
vcat(mean(as1, dims=1), std(as1, dims=1))